In [ ]:
from Functions import *

In [ ]:
# 尾盘集合竞价实际时间是14:57 - 15:00
# 尾盘集合竞价时期Tick数据
def update_stock_tail_call_tick_data(date):
    name = "StockTailCallTick"
    local_file = f'/home/ubuntu/Lwy_research/ITFiles/{name}_{date}.fea'

    # 读取当天的local_tick数据
    total_data = pd.read_feather(rf"./sample_data/local_tick_sample_{date}.fea")

    # 做好时间截取，然后去掉一些完全没有用的字段
    local_data = total_data[total_data['time']>=145700000]
    local_data = local_data.drop(['open', 'high', 'low'], axis=1)
    local_data = local_data.drop(['cjbs', 'cjbs_sum', 'volume_sum', 'amount_sum'], axis=1)
    for level in range(3, 11):
        local_data = local_data.drop([f'bp{level}', f'sp{level}', f'bv{level}', f'sv{level}'], axis=1)
    local_data = local_data.drop(['bp_avg', 'sp_avg', 'bv_sum', 'sv_sum'], axis=1)
    local_data = local_data.reset_index(drop=True)
    # 输出数据
    local_data.to_feather(local_file, compression='zstd')
    print(local_file)
    
date = "20240730"
update_stock_tail_call_tick_data(date)

In [ ]:
# LastTrans数据：进入尾盘集合竞价阶段之前的最后一笔成交信息，在制作Order中需要用到
def update_stock_tail_call_last_trans_data(date):
    try:
        name = "StockTailCallLastTrans"
        local_file = f'/home/ubuntu/Lwy_research/ITFiles/{name}_{date}.fea'

        # 直接读取全天的数据，因为都是已经整理好了的，格式上不需要再做任何调整
        trans_data = pd.read_feather(rf"./sample_data/local_trans_sample_{date}.fea")

        # 截取出收盘集合竞价之前的最后一笔成交信息
        call_last_trans_data = trans_data[trans_data['time']<=145900000] # 因为Total中已经不包含撤单数据，所以不用对transType做任何判断
        call_last_trans_data = call_last_trans_data.drop_duplicates(subset=['code'], keep='last')
        local_data = call_last_trans_data.reset_index(drop=True)
        # 输出数据
        local_data.to_feather(local_file, compression='zstd')
        del trans_data

    except Exception as e:
        log_info(name, date, str(e))

for date in get_trade_days("20240726", "20240731"):
    update_stock_tail_call_last_trans_data(date)

In [ ]:
# Order数据, 尾盘集合竞价order数据由两部分构成：集合竞价开始之前没有成交的委托 + 集合竞价开始之后才挂单的委托
# 集合竞价开始之前的委托计算：需要去除已成交和撤单的部分
def update_stock_tail_call_order_data(date):

    def get_order_by_code(code, order_data_dict, trans_data_dict, last_price_dict):
        if code not in last_price_dict.keys(): # 如果给不出lastprice，不计算order了，没有意义
            return pd.DataFrame()
        
        last_price = last_price_dict[code]
        order_data = order_data_dict[code]
        order_data_tail = order_data[order_data['time']>=145700000] # 集合竞价开始之后的委托，都保留
        order_data_left = order_data[order_data['time']<145700000] # 集合竞价开始之前的委托，需要剔除

        if code in trans_data_dict.keys(): # 有trans数据的话就剔除order中已成交/撤单的部分
            trans_data = trans_data_dict[code]
            trans_data = trans_data[trans_data['time']<145700000] # 取集合竞价开始之前的部分
            trans_B_volume = trans_data[trans_data["bidId"]>0].groupby("bidId")["transVolume"].sum()
            trans_S_volume = trans_data[trans_data["askId"]>0].groupby("askId")["transVolume"].sum()
            trans_volume = trans_B_volume.append(trans_S_volume)

            # 把left里面的原始委托编号sum起来，再减去trans_volume，就是这些委托单的净剩余量
            order_data_left_volume = order_data_left.groupby(["orderIdOrigin"])["orderVolume"].sum()
            order_data_left_volume -= trans_volume.reindex_like(order_data_left_volume).fillna(0)
            order_data_left["orderVolume"] = order_data_left["orderIdOrigin"].map(order_data_left_volume)
            order_data_left = order_data_left[order_data_left["orderVolume"]>0] # 只留下还有委托量的订单
        
        # 和tail数据结合起来，形成总的数据
        local_data_sub = order_data_left.append(order_data_tail)
        local_data_sub = local_data_sub.sort_values('orderId') # 按照委托编号进行严格排序
        local_data_sub["lastprice"] = last_price # 拼接一个lastprice进去
        local_data_sub['orderVolume'] = round(local_data_sub['orderVolume'] + 1e-8).astype(np.int64) # volume还是取int

        return local_data_sub  
    
    try:
        name = "StockTailCallOrder"
        local_file = f'/home/ubuntu/Lwy_research/ITFiles/{name}_{date}.fea'

        total_order_data = pd.read_feather(rf"./sample_data/local_order_sample_{date}.fea")  
        total_trans_data = pd.read_feather(rf"./sample_data/local_trans_sample_{date}.fea")            
        order_data_dict = change_df2dict(total_order_data) # 读取全天的委托数据
        trans_data_dict = change_df2dict(total_trans_data) # 读取全天的成交数据
        last_price_dict = pd.read_feather(rf"./StockTailCallLastTransData_{date}.fea").set_index("code")["transPrice"].to_dict()

        code_list = list(order_data_dict.keys())
        code_list.sort()

        local_data_list = []
        for code in code_list:
            local_data_sub = get_order_by_code(code, order_data_dict, trans_data_dict, last_price_dict)
            local_data_list.append(local_data_sub)
        local_data = pd.concat(local_data_list).reset_index(drop=True)
        del order_data_dict, trans_data_dict, last_price_dict

        # 输出数据
        local_data.to_feather(local_file, compression='zstd')

    except Exception as e:
        log_info(name, date, str(e))
for date in get_trade_days("20240726", "20240731"):
    update_stock_tail_call_order_data(date)